

# 1.0 Multiple hypothesis testing



In this notebook we will study a method of correcting for multiple hypothesis testing, which occurs often in practice when searching for anomalies in multiple datasets.

Let's start from the basics and illustrate what does it mean to have multiple tests.

Roughly speaking, a statistical test is something capable of either **rejecting** or **not the null hypothesis** with a given type I error probability $\alpha$ (probability of a false positive) and a type II error probability $\beta$ (probability of a false negative).

If you need a refresher, look in your favorite textbook or on [Wikipedia](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors).

## 1.1 Our toy problem



Let's consider the problem of determining whether two populations have the same average or not. The **null hypothesis** is that the average is the same, the alternative is that it is not. 

Note that you can substitute this problem with any other problem answerable with a statistical test and all the discussion here will still hold (but of course you need to rework the code).

## 1.2 Simple case: one test

The test appropriate for the problem at hand is the [Student's T-test](https://en.wikipedia.org/wiki/Student%27s_t-test). Let's write the function that computes the p-value and a function that decides whether the null is rejected or not based on the p-value:

In [17]:
import numpy as np
import scipy.stats

# Let's write so that w1 and w2 can be lists of n
# datasets, from 1 to as much as needed
def apply_ttest(w1, w2):
    
    ts, pvalues = scipy.stats.ttest_ind(w1, w2, axis=1)

    return np.squeeze(pvalues)

# The null is accepted if all the pvalues are larger 
# than alpha (global null hypothesis, see below)
def null_hyp_status(pvalue, alpha):
    
    return np.all(pvalue > alpha)

In [18]:
def generate_dataset(n_datasets, n_null_true, n_samples=100, seed=0):
    
    # This is to make the results predictable
    np.random.seed(seed)
    
    n_null_false = n_datasets - n_null_true
    
    w1 = []
    w2 = []
    null_status = []
    
    for i in range(n_null_true):
        
        wn_1 = np.random.normal(loc=90, scale=10, size=n_samples)
        wn_2 = np.random.normal(loc=90, scale=10, size=n_samples)
    
        w1.append(wn_1)
        w2.append(wn_2)
        
        null_status.append(True)
    
    for i in range(n_null_false):
        
        wn_1 = np.random.normal(loc=95, scale=10, size=n_samples)
        wn_2 = np.random.normal(loc=90, scale=10, size=n_samples)
    
        w1.append(wn_1)
        w2.append(wn_2)
        null_status.append(False)
    
    return w1, w2, np.array(null_status)

We now generate 1 dataset with the null hypothesis true, and apply the test. We will use a type I error probability $\alpha=0.05$:

In [63]:
# Let's get a dataset with 1 group and
# the null hypothesis is true
w1, w2, ground_truth = generate_dataset(n_datasets=1, n_null_true=1)

# Let's now apply the test
alpha = 0.05
pvalue = apply_ttest(w1, w2)

print("Null hyp. is deemed %s (p_value = %f)" % (null_hyp_status(pvalue, alpha),pvalue))

Null hyp. is deemed True (p_value = 0.878587)


The test worked as expected, and didn't reject the null hypothesis (which we know is true). Let's verify that the performance of the test is nominal, i.e., that by repeating a large number of independent realizations of the same experiment we reject by chance the null hypothesis with the nominal type I error probability $\alpha$:

In [64]:
# Let's now apply the test
alpha = 0.05

# Times the test was rejected
rejected = 0

# Number of iterations
iter = 5000

for i in range(iter):
    seed = (i+1) * 1000
    w1, w2, ground_truth = generate_dataset(n_datasets=1, n_null_true=1, seed=seed)

    pvalue = apply_ttest(w1, w2)

    if not null_hyp_status(pvalue, alpha):
        rejected += 1

print("\nMeasured chance probability of rejecting the "
      "null: %.3f (should be %.3f)" % (rejected/iter, alpha))


Measured chance probability of rejecting the null: 0.052 (should be 0.050)


ok, it works as expected.


## 1.3 Multiple tests



Let's now imagine that we have $m$ pairs of populations, and we want to find out whether one or more pairs have a significant difference between the populations.

- The null hypothesis here is "within all pairs, the two populations have the same average".
- The alternative one is "there is at least one pair where the average is different between the two populations".

Can we just apply the test separately to each pair and see if it rejects for at least one? (spoiler: the answer is no! Also, let's neglect the fact that there are other tests designed for this situation, like ANOVA). Let's see:

In [66]:
# Generate m=50 pairs of populations, all with the same
# average between the populations (the null hypothesis is true)
w1, w2, _ = generate_dataset(n_datasets=50, n_null_true=50)
pvalues = apply_ttest(w1, w2)

print("Null hyp. is deemed %s" % null_hyp_status(pvalues, alpha))

Null hyp. is deemed False


At first this result might come as a suprise. After all, we know that the null hypothesis is true!

However, if you recall the definition of Type I error probability, by fixing $\alpha=0.05$ we are setting up the test so that it will wrongly reject the null with 5% probability. Therefore, by repeating the test 50 times (one for each pair) we had each time a 5% chance of a type I error. The probability of having at least a rejection is hence given by the [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution):

In [67]:
# probability of having one or more rejections in 50 trials
m = 50

binomial_distr = scipy.stats.binom(m, alpha)

# NOTE:  gives the probability cumulative of obtaining <= 0,
# while we need >= 1, so we sub 1 - CDF(0)
prob = 1-binomial_distr.cdf(0)

print("The prob. of >= 1 false positives in %i "
      "trials is %.3f" % (m, prob))

The prob. of >= 1 false positives in 50 trials is 0.923


There is over 90% chance to get at least one false positive in our setup. Testing multiple times an hypothesis as part of the same question is called "multiple testing" and require some more thoughts.



## 1.4 Bonferroni / Sidak correction



Bonferroni (1936) introduced a simple correction for situations like this. The prescription is to substitute $\alpha$ for each one of the $m$ independent tests within the composite test with a corrected type I error probability given by the Sidak formula $\alpha^{\prime} = 1 - (1-\alpha)^{1/m}$ (which for large $m$ is often approximated with $\alpha^{\prime} = \alpha / m$). 

> Sometimes in the literature the correction $\alpha^{\prime}=\alpha / m$ is called "Bonferroni correction" while the correction $\alpha^{\prime} = 1 - (1-\alpha)^{1/m}$ is called "Sidak correction". Here we will use the latter formulation, but use the name interceangably as the difference is very small for all practical purposes

> NOTE: we assume independent tests. If there is correlation between the different tests, the methods presented here might or might not apply, you need to look closer at the relevant papers.

Let's see if this solves our problem. We just need to change the criterium used to decide whether to reject or not the null, no need to change the computation of the p-values:

In [68]:
# Test if any of the pvalues is lower than alpha',
# if the answer yes, the null hyp. is deemed False
def null_hyp_status_bonferroni(pvalues, alpha):
    
    # Number of tests
    m = pvalues.shape[0]
    
    # Bonferroni/Sidak correction
    alpha_prime = 1 - (1-alpha)**(1.0/m)
    
    # Test whether *all* null hypothesis in the subtests are
    # true or not
    return np.all(pvalues > alpha_prime)

w1, w2, _ = generate_dataset(n_datasets=50, n_null_true=50)
pvalues = apply_ttest(w1, w2)

print("Null hyp. is deemed %s" % null_hyp_status_bonferroni(pvalues, alpha))

Null hyp. is deemed True


That looks better. In order to make sure, let's generate a lot of synthetic datasets as earlier and let's see if our Bonferroni-corrected test provides the nominal type I error probability $\alpha=0.05$:

In [69]:
# Let's now apply the test
alpha = 0.05

# Times the test was rejected
rejected = 0

# Number of iterations
iter = 5000

for i in range(iter):
    seed = (i+1) * 1000
    w1, w2, ground_truth = generate_dataset(n_datasets=50, n_null_true=50, seed=seed)

    pvalue = apply_ttest(w1, w2)

    if not null_hyp_status_bonferroni(pvalue, alpha):
        rejected += 1

print("\nMeasured chance probability of rejecting the "
      "null: %.3f (should be %.3f)" % (rejected/iter, alpha))


Measured chance probability of rejecting the null: 0.047 (should be 0.050)


It worked. The type I error probability is indeed very close to the nominal 5%.